# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments              Customer  \
0  Nov  2 2022  4:41PM  249933        15  PNC401085A  Person & Covey, Inc.   
1  Nov  2 2022  4:41PM  249933        15  PNC401086A  Person & Covey, Inc.   
2  Nov  2 2022  4:41PM  249933        15  PNC401089A  Person & Covey, Inc.   
3  Nov  2 2022  4:41PM  249933        15  PNC401091A  Person & Covey, Inc.   
4  Nov  2 2022  4:41PM  249933        15  PNC401096A  Person & Covey, Inc.   
5  Nov  2 2022  4:41PM  249933        15  PNC401097A  Person & Covey, Inc.   
6  Nov  2 2022  4:41PM  249933        15  PNC401101A  Person & Covey, Inc.   
7  Nov  2 2022  4:41PM  249933        15  PNC401103A  Person & Covey, Inc.   
8  Nov  2 2022  4:41PM  249933        15  PNC401112A  Person & Covey, Inc.   
9  Nov  2 2022  4:41PM  249933        15  PNC401113A  Person & Covey, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
21  249927       Released          5
22  249928       Released          5
23  249929       Released          3
24  249930       Released         16
25  249933       Released         28

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
249927                NaN        NaN       5.0
249928                NaN        NaN       5.0
249929                NaN        NaN       3.0
249930                NaN        NaN      16.0
249933                NaN        NaN      28.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249856               26.0        1.0       1.0
249880                0.0        0.0       1.0
249881                0.0        0.0       1.0
249884                0.0        0.0       1.0
249885                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249856                 26          1         1
249880                  0          0         1
249881                  0          0         1
249884                  0          0         1
249885                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               249856         26          1         1
1               249880          0          0         1
2               249881          0          0         1
3               249884          0          0         1
4               249885          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               249856        26         1         1
1               249880                             1
2               249881                             1
3               249884                             1
4               249885                             1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Nov  2 2022  4:41PM  249933        15             Person & Covey, Inc.
28  Nov  2 2022  4:11PM  249930        19              AdvaGen Pharma, Ltd
44  Nov  2 2022  4:03PM  249929        19                   Innogenix, LLC
47  Nov  2 2022  3:57PM  249928        19                   Innogenix, LLC
52  Nov  2 2022  3:48PM  249927        19                   Innogenix, LLC
57  Nov  2 2022  3:40PM  249926        19                   Innogenix, LLC
62  Nov  2 2022  3:15PM  249924        19              AdvaGen Pharma, Ltd
84  Nov  2 2022  1:41PM  249911        21                NBTY Global, Inc.
85  Nov  2 2022  1:30PM  249906        21                NBTY Global, Inc.
86  Nov  2 2022 12:48PM  249900        20  HVL, LLC dba Atrium Innovations

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Nov  2 2022  4:41PM  249933        15             Person & Covey, Inc.   
1  Nov  2 2022  4:11PM  249930        19              AdvaGen Pharma, Ltd   
2  Nov  2 2022  4:03PM  249929        19                   Innogenix, LLC   
3  Nov  2 2022  3:57PM  249928        19                   Innogenix, LLC   
4  Nov  2 2022  3:48PM  249927        19                   Innogenix, LLC   
5  Nov  2 2022  3:40PM  249926        19                   Innogenix, LLC   
6  Nov  2 2022  3:15PM  249924        19              AdvaGen Pharma, Ltd   
7  Nov  2 2022  1:41PM  249911        21                NBTY Global, Inc.   
8  Nov  2 2022  1:30PM  249906        21                NBTY Global, Inc.   
9  Nov  2 2022 12:48PM  249900        20  HVL, LLC dba Atrium Innovations   

  Completed Executing  Released  
0                            28  
1                            16  
2                             3  
3                             5  
4                             5  
5                             5  
6                  10        12  
7                             1  
8                             1  
9                             1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Nov  2 2022  4:41PM  249933        15             Person & Covey, Inc.   
1  Nov  2 2022  4:11PM  249930        19              AdvaGen Pharma, Ltd   
2  Nov  2 2022  4:03PM  249929        19                   Innogenix, LLC   
3  Nov  2 2022  3:57PM  249928        19                   Innogenix, LLC   
4  Nov  2 2022  3:48PM  249927        19                   Innogenix, LLC   
5  Nov  2 2022  3:40PM  249926        19                   Innogenix, LLC   
6  Nov  2 2022  3:15PM  249924        19              AdvaGen Pharma, Ltd   
7  Nov  2 2022  1:41PM  249911        21                NBTY Global, Inc.   
8  Nov  2 2022  1:30PM  249906        21                NBTY Global, Inc.   
9  Nov  2 2022 12:48PM  249900        20  HVL, LLC dba Atrium Innovations   

   Released Executing Completed  
0        28                      
1        16                      
2         3                      
3         5                      
4         5                      
5         5                      
6        12        10            
7         1                      
8         1                      
9         1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse              Customer  Released  \
0  Nov  2 2022  4:41PM  249933        15  Person & Covey, Inc.        28   
1  Nov  2 2022  4:11PM  249930        19   AdvaGen Pharma, Ltd        16   
2  Nov  2 2022  4:03PM  249929        19        Innogenix, LLC         3   
3  Nov  2 2022  3:57PM  249928        19        Innogenix, LLC         5   
4  Nov  2 2022  3:48PM  249927        19        Innogenix, LLC         5   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse              Customer  Released  \
0  Nov  2 2022  4:41PM  249933        15  Person & Covey, Inc.        28   
1  Nov  2 2022  4:11PM  249930        19   AdvaGen Pharma, Ltd        16   
2  Nov  2 2022  4:03PM  249929        19        Innogenix, LLC         3   
3  Nov  2 2022  3:57PM  249928        19        Innogenix, LLC         5   
4  Nov  2 2022  3:48PM  249927        19        Innogenix, LLC         5   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse              Customer  Released  \
0  Nov  2 2022  4:41PM  249933        15  Person & Covey, Inc.        28   
1  Nov  2 2022  4:11PM  249930        19   AdvaGen Pharma, Ltd        16   
2  Nov  2 2022  4:03PM  249929        19        Innogenix, LLC         3   
3  Nov  2 2022  3:57PM  249928        19        Innogenix, LLC         5   
4  Nov  2 2022  3:48PM  249927        19        Innogenix, LLC         5   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
15          499789        29        1.0       26.0
16          499772         2        0.0        0.0
19         1499564        46       10.0        0.0
20          249900         1        0.0        0.0
21         2998709        12        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        15   499789        29        1.0       26.0
1        16   499772         2        0.0        0.0
2        19  1499564        46       10.0        0.0
3        20   249900         1        0.0        0.0
4        21  2998709        12        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        15        29        1.0       26.0
1        16         2        0.0        0.0
2        19        46       10.0        0.0
3        20         1        0.0        0.0
4        21        12        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        15  Released   29.0
1        16  Released    2.0
2        19  Released   46.0
3        20  Released    1.0
4        21  Released   12.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    15   16    19   20    21
Status                               
Completed  26.0  0.0   0.0  0.0   0.0
Executing   1.0  0.0  10.0  0.0   0.0
Released   29.0  2.0  46.0  1.0  12.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    15   16    19   20    21
0          Completed  26.0  0.0   0.0  0.0   0.0
1          Executing   1.0  0.0  10.0  0.0   0.0
2           Released  29.0  2.0  46.0  1.0  12.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    15   16    19   20    21
0  Completed  26.0  0.0   0.0  0.0   0.0
1  Executing   1.0  0.0  10.0  0.0   0.0
2   Released  29.0  2.0  46.0  1.0  12.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()